In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 13.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [2]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
            unattached_dfs.append(process(pd.read_csv(address)))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")     

In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [19]:
clmns = list(range(1, 21))
clmns.append("suggestion")
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10, 10))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i):
  mean_of_next = []
  for ix in range(i,i+10):
    mean_of_next.append(data[ix][0]+data[ix][3])
  mean_of_next = [element / 2 for element in mean_of_next]
  mean_of_next = sum(mean_of_next)/5

  mean_of_previous = []
  for ix in range(i-10,i):
    mean_of_previous.append(data[ix][0]+data[ix][3])
  mean_of_previous = [element / 2 for element in mean_of_previous]
  mean_of_previous = sum(mean_of_previous)/5

  if mean_of_next > mean_of_previous:
    return "buy"
  else:
    return "sell"

def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(100, data.shape[0]-10):
        opens  = []
        highs  = []
        lows   = []
        closes = []

        opens.append([
                        (data[i-100][0]+data[i-99][0]+data[i-98][0]+data[i-97][0]+data[i-96][0])/5 -  (data[i-95][0]+data[i-94][0]+data[i-93][0]+data[i-92][0]+data[i-91][0])/5,
                        (data[i-90][0]+data[i-89][0]+data[i-88][0]+data[i-87][0]+data[i-86][0])/5 -  (data[i-85][0]+data[i-84][0]+data[i-83][0]+data[i-82][0]+data[i-81][0])/5,
                        (data[i-80][0]+data[i-79][0]+data[i-78][0]+data[i-77][0]+data[i-76][0])/5 -  (data[i-75][0]+data[i-74][0]+data[i-73][0]+data[i-72][0]+data[i-71][0])/5,
                        (data[i-70][0]+data[i-69][0]+data[i-68][0]+data[i-67][0]+data[i-66][0])/5 -  (data[i-65][0]+data[i-64][0]+data[i-63][0]+data[i-62][0]+data[i-61][0])/5,
                        (data[i-60][0]+data[i-59][0]+data[i-58][0]+data[i-57][0]+data[i-56][0])/5 -  (data[i-55][0]+data[i-54][0]+data[i-53][0]+data[i-52][0]+data[i-51][0])/5,
                        (data[i-50][0]+data[i-49][0]+data[i-48][0]+data[i-47][0]+data[i-46][0])/5 -  (data[i-45][0]+data[i-44][0]+data[i-43][0]+data[i-42][0]+data[i-41][0])/5,
                        (data[i-40][0]+data[i-39][0]+data[i-38][0]+data[i-37][0]+data[i-36][0])/5 -  (data[i-35][0]+data[i-34][0]+data[i-33][0]+data[i-32][0]+data[i-31][0])/5,
                        (data[i-30][0]+data[i-29][0]+data[i-28][0]+data[i-27][0]+data[i-26][0])/5 -  (data[i-25][0]+data[i-24][0]+data[i-23][0]+data[i-22][0]+data[i-21][0])/5,
                        (data[i-20][0]+data[i-19][0]+data[i-18][0]+data[i-17][0]+data[i-16][0])/5 -  (data[i-15][0]+data[i-14][0]+data[i-13][0]+data[i-12][0]+data[i-11][0])/5,
                        (data[i-10][0]+data[i-9][0]+data[i-8][0]+data[i-7][0]+data[i-6][0])/5     -  (data[i-5][0]+data[i-4][0]+data[i-3][0]+data[i-2][0]+data[i-1][0])/5])
        closes.append([
                        (data[i-100][3]+data[i-99][3]+data[i-98][3]+data[i-97][3]+data[i-96][3])/5 -  (data[i-95][3]+data[i-94][3]+data[i-93][3]+data[i-92][3]+data[i-91][3])/5,
                        (data[i-90][3]+data[i-89][3]+data[i-88][3]+data[i-87][3]+data[i-86][3])/5 -  (data[i-85][3]+data[i-84][3]+data[i-83][3]+data[i-82][3]+data[i-81][3])/5, 
                        (data[i-80][3]+data[i-79][3]+data[i-78][3]+data[i-77][3]+data[i-76][3])/5 -  (data[i-75][3]+data[i-74][3]+data[i-73][3]+data[i-72][3]+data[i-71][3])/5,
                        (data[i-70][3]+data[i-69][3]+data[i-68][3]+data[i-67][3]+data[i-66][3])/5 -  (data[i-65][3]+data[i-64][3]+data[i-63][3]+data[i-62][3]+data[i-61][3])/5,
                        (data[i-60][3]+data[i-59][3]+data[i-58][3]+data[i-57][3]+data[i-56][3])/5 -  (data[i-55][3]+data[i-54][3]+data[i-53][3]+data[i-52][3]+data[i-51][3])/5,
                        (data[i-50][3]+data[i-49][3]+data[i-48][3]+data[i-47][3]+data[i-46][3])/5 -  (data[i-45][3]+data[i-44][3]+data[i-43][3]+data[i-42][3]+data[i-41][3])/5,
                        (data[i-40][3]+data[i-39][3]+data[i-38][3]+data[i-37][3]+data[i-36][3])/5 -  (data[i-35][3]+data[i-34][3]+data[i-33][3]+data[i-32][3]+data[i-31][3])/5,
                        (data[i-30][3]+data[i-29][3]+data[i-28][3]+data[i-27][3]+data[i-26][3])/5 -  (data[i-25][3]+data[i-24][3]+data[i-23][3]+data[i-22][3]+data[i-21][3])/5,
                        (data[i-20][3]+data[i-19][3]+data[i-18][3]+data[i-17][3]+data[i-16][3])/5 -  (data[i-15][3]+data[i-14][3]+data[i-13][3]+data[i-12][3]+data[i-11][3])/5,
                        (data[i-10][3]+data[i-9][3]+data[i-8][3]+data[i-7][3]+data[i-6][3])/5     -  (data[i-5][3]+data[i-4][3]+data[i-3][3]+data[i-2][3]+data[i-1][3])/5])

        sugg = make_y(data, i)
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)


In [20]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1499
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240  
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

In [16]:
yTrain.value_counts()

suggestion_buy  suggestion_sell
1               0                  47006
0               1                  45451
dtype: int64

In [21]:
model = Sequential()

model.add(Dense(750, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 750)               15750     
                                                                 
 dense_16 (Dense)            (None, 750)               563250    
                                                                 
 dense_17 (Dense)            (None, 750)               563250    
                                                                 
 dense_18 (Dense)            (None, 750)               563250    
                                                                 
 dense_19 (Dense)            (None, 2)                 1502      
                                                                 
Total params: 1,707,002
Trainable params: 1,707,002
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=25,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"MCG136_{acr}.h5")

Epoch 1/25
2890/2890 [==============================] - 65s 22ms/step - loss: 0.4155 - accuracy: 0.8148 - val_loss: 0.3606 - val_accuracy: 0.8414
Epoch 2/25
2890/2890 [==============================] - 63s 22ms/step - loss: 0.3293 - accuracy: 0.8574 - val_loss: 0.3034 - val_accuracy: 0.8704
Epoch 3/25
2890/2890 [==============================] - 62s 22ms/step - loss: 0.2560 - accuracy: 0.8931 - val_loss: 0.2391 - val_accuracy: 0.8980
Epoch 4/25
2890/2890 [==============================] - 63s 22ms/step - loss: 0.2017 - accuracy: 0.9187 - val_loss: 0.2116 - val_accuracy: 0.9128
Epoch 5/25
2890/2890 [==============================] - 62s 22ms/step - loss: 0.1653 - accuracy: 0.9335 - val_loss: 0.1697 - val_accuracy: 0.9345
Epoch 6/25
2890/2890 [==============================] - 63s 22ms/step - loss: 0.1414 - accuracy: 0.9441 - val_loss: 0.1602 - val_accuracy: 0.9409
Epoch 7/25
1941/2890 [===================>..........] - ETA: 19s - loss: 0.1226 - accuracy: 0.9522

In [123]:
def process_for_prediction(data,index):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        opens = []
        highs = []
        lows = []
        closes = []
        opens.append([
                        (data[i-100][0]+data[i-99][0]+data[i-98][0]+data[i-97][0]+data[i-96][0])/5 -  (data[i-95][0]+data[i-94][0]+data[i-93][0]+data[i-92][0]+data[i-91][0])/5,
                        (data[i-90][0]+data[i-89][0]+data[i-88][0]+data[i-87][0]+data[i-86][0])/5 -  (data[i-85][0]+data[i-84][0]+data[i-83][0]+data[i-82][0]+data[i-81][0])/5,
                        (data[i-80][0]+data[i-79][0]+data[i-78][0]+data[i-77][0]+data[i-76][0])/5 -  (data[i-75][0]+data[i-74][0]+data[i-73][0]+data[i-72][0]+data[i-71][0])/5,
                        (data[i-70][0]+data[i-69][0]+data[i-68][0]+data[i-67][0]+data[i-66][0])/5 -  (data[i-65][0]+data[i-64][0]+data[i-63][0]+data[i-62][0]+data[i-61][0])/5,
                        (data[i-60][0]+data[i-59][0]+data[i-58][0]+data[i-57][0]+data[i-56][0])/5 -  (data[i-55][0]+data[i-54][0]+data[i-53][0]+data[i-52][0]+data[i-51][0])/5,
                        (data[i-50][0]+data[i-49][0]+data[i-48][0]+data[i-47][0]+data[i-46][0])/5 -  (data[i-45][0]+data[i-44][0]+data[i-43][0]+data[i-42][0]+data[i-41][0])/5,
                        (data[i-40][0]+data[i-39][0]+data[i-38][0]+data[i-37][0]+data[i-36][0])/5 -  (data[i-35][0]+data[i-34][0]+data[i-33][0]+data[i-32][0]+data[i-31][0])/5,
                        (data[i-30][0]+data[i-29][0]+data[i-28][0]+data[i-27][0]+data[i-26][0])/5 -  (data[i-25][0]+data[i-24][0]+data[i-23][0]+data[i-22][0]+data[i-21][0])/5,
                        (data[i-20][0]+data[i-19][0]+data[i-18][0]+data[i-17][0]+data[i-16][0])/5 -  (data[i-15][0]+data[i-14][0]+data[i-13][0]+data[i-12][0]+data[i-11][0])/5,
                        (data[i-10][0]+data[i-9][0]+data[i-8][0]+data[i-7][0]+data[i-6][0])/5     -  (data[i-5][0]+data[i-4][0]+data[i-3][0]+data[i-2][0]+data[i-1][0])/5])
        highs.append([
                        (data[i-100][1]+data[i-99][1]+data[i-98][1]+data[i-97][1]+data[i-96][1])/5 -  (data[i-95][1]+data[i-94][1]+data[i-93][1]+data[i-92][1]+data[i-91][1])/5,
                        (data[i-90][1]+data[i-89][1]+data[i-88][1]+data[i-87][1]+data[i-86][1])/5 -  (data[i-85][1]+data[i-84][1]+data[i-83][1]+data[i-82][1]+data[i-81][1])/5,
                        (data[i-80][1]+data[i-79][1]+data[i-78][1]+data[i-77][1]+data[i-76][1])/5 -  (data[i-75][1]+data[i-74][1]+data[i-73][1]+data[i-72][1]+data[i-71][1])/5,
                        (data[i-70][1]+data[i-69][1]+data[i-68][1]+data[i-67][1]+data[i-66][1])/5 -  (data[i-65][1]+data[i-64][1]+data[i-63][1]+data[i-62][1]+data[i-61][1])/5,
                        (data[i-60][1]+data[i-59][1]+data[i-58][1]+data[i-57][1]+data[i-56][1])/5 -  (data[i-55][1]+data[i-54][1]+data[i-53][1]+data[i-52][1]+data[i-51][1])/5,
                        (data[i-50][1]+data[i-49][1]+data[i-48][1]+data[i-47][1]+data[i-46][1])/5 -  (data[i-45][1]+data[i-44][1]+data[i-43][1]+data[i-42][1]+data[i-41][1])/5,
                        (data[i-40][1]+data[i-39][1]+data[i-38][1]+data[i-37][1]+data[i-36][1])/5 -  (data[i-35][1]+data[i-34][1]+data[i-33][1]+data[i-32][1]+data[i-31][1])/5,
                        (data[i-30][1]+data[i-29][1]+data[i-28][1]+data[i-27][1]+data[i-26][1])/5 -  (data[i-25][1]+data[i-24][1]+data[i-23][1]+data[i-22][1]+data[i-21][1])/5,
                        (data[i-20][1]+data[i-19][1]+data[i-18][1]+data[i-17][1]+data[i-16][1])/5 -  (data[i-15][1]+data[i-14][1]+data[i-13][1]+data[i-12][1]+data[i-11][1])/5,
                        (data[i-10][1]+data[i-9][1]+data[i-8][1]+data[i-7][1]+data[i-6][1])/5     -  (data[i-5][1]+data[i-4][1]+data[i-3][1]+data[i-2][1]+data[i-1][1])/5])
        lows.append([
                        (data[i-100][2]+data[i-99][2]+data[i-98][2]+data[i-97][2]+data[i-96][2])/5 -  (data[i-95][2]+data[i-94][2]+data[i-93][2]+data[i-92][2]+data[i-91][2])/5,
                        (data[i-90][2]+data[i-89][2]+data[i-88][2]+data[i-87][2]+data[i-86][2])/5 -  (data[i-85][2]+data[i-84][2]+data[i-83][2]+data[i-82][2]+data[i-81][2])/5,
                        (data[i-80][2]+data[i-79][2]+data[i-78][2]+data[i-77][2]+data[i-76][2])/5 -  (data[i-75][2]+data[i-74][2]+data[i-73][2]+data[i-72][2]+data[i-71][2])/5,
                        (data[i-70][2]+data[i-69][2]+data[i-68][2]+data[i-67][2]+data[i-66][2])/5 -  (data[i-65][2]+data[i-64][2]+data[i-63][2]+data[i-62][2]+data[i-61][2])/5,
                        (data[i-60][2]+data[i-59][2]+data[i-58][2]+data[i-57][2]+data[i-56][2])/5 -  (data[i-55][2]+data[i-54][2]+data[i-53][2]+data[i-52][2]+data[i-51][2])/5,
                        (data[i-50][2]+data[i-49][2]+data[i-48][2]+data[i-47][2]+data[i-46][2])/5 -  (data[i-45][2]+data[i-44][2]+data[i-43][2]+data[i-42][2]+data[i-41][2])/5,
                        (data[i-40][2]+data[i-39][2]+data[i-38][2]+data[i-37][2]+data[i-36][2])/5 -  (data[i-35][2]+data[i-34][2]+data[i-33][2]+data[i-32][2]+data[i-31][2])/5,
                        (data[i-30][2]+data[i-29][2]+data[i-28][2]+data[i-27][2]+data[i-26][2])/5 -  (data[i-25][2]+data[i-24][2]+data[i-23][2]+data[i-22][2]+data[i-21][2])/5,
                        (data[i-20][2]+data[i-19][2]+data[i-18][2]+data[i-17][2]+data[i-16][2])/5 -  (data[i-15][2]+data[i-14][2]+data[i-13][2]+data[i-12][2]+data[i-11][2])/5,
                        (data[i-10][2]+data[i-9][2]+data[i-8][2]+data[i-7][2]+data[i-6][2])/5     -  (data[i-5][2]+data[i-4][2]+data[i-3][2]+data[i-2][2]+data[i-1][2])/5])
        closes.append([
                        (data[i-100][3]+data[i-99][3]+data[i-98][3]+data[i-97][3]+data[i-96][3])/5 -  (data[i-95][3]+data[i-94][3]+data[i-93][3]+data[i-92][3]+data[i-91][3])/5,
                        (data[i-90][3]+data[i-89][3]+data[i-88][3]+data[i-87][3]+data[i-86][3])/5 -  (data[i-85][3]+data[i-84][3]+data[i-83][3]+data[i-82][3]+data[i-81][3])/5, 
                        (data[i-80][3]+data[i-79][3]+data[i-78][3]+data[i-77][3]+data[i-76][3])/5 -  (data[i-75][3]+data[i-74][3]+data[i-73][3]+data[i-72][3]+data[i-71][3])/5,
                        (data[i-70][3]+data[i-69][3]+data[i-68][3]+data[i-67][3]+data[i-66][3])/5 -  (data[i-65][3]+data[i-64][3]+data[i-63][3]+data[i-62][3]+data[i-61][3])/5,
                        (data[i-60][3]+data[i-59][3]+data[i-58][3]+data[i-57][3]+data[i-56][3])/5 -  (data[i-55][3]+data[i-54][3]+data[i-53][3]+data[i-52][3]+data[i-51][3])/5,
                        (data[i-50][3]+data[i-49][3]+data[i-48][3]+data[i-47][3]+data[i-46][3])/5 -  (data[i-45][3]+data[i-44][3]+data[i-43][3]+data[i-42][3]+data[i-41][3])/5,
                        (data[i-40][3]+data[i-39][3]+data[i-38][3]+data[i-37][3]+data[i-36][3])/5 -  (data[i-35][3]+data[i-34][3]+data[i-33][3]+data[i-32][3]+data[i-31][3])/5,
                        (data[i-30][3]+data[i-29][3]+data[i-28][3]+data[i-27][3]+data[i-26][3])/5 -  (data[i-25][3]+data[i-24][3]+data[i-23][3]+data[i-22][3]+data[i-21][3])/5,
                        (data[i-20][3]+data[i-19][3]+data[i-18][3]+data[i-17][3]+data[i-16][3])/5 -  (data[i-15][3]+data[i-14][3]+data[i-13][3]+data[i-12][3]+data[i-11][3])/5,
                        (data[i-10][3]+data[i-9][3]+data[i-8][3]+data[i-7][3]+data[i-6][3])/5     -  (data[i-5][3]+data[i-4][3]+data[i-3][3]+data[i-2][3]+data[i-1][3])/5])

        mean_of_previous = []
        for ix in range(i-5,i):
           mean_of_previous.append(data[ix][0]+data[ix][3])
        mean_of_previous = [element / 2 for element in mean_of_previous]
        mean_of_previous = sum(mean_of_previous)/5
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1))
        return arr,mean_of_previous
def make_prediction_for_yf(symbol,period,timeframe,index):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index)
    return f"""YF   : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=105), tindex)
    return f"""TVB  : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex))

In [124]:
make_prediction("btc-usd","max","1wk","btcusd","bitstamp",Interval.in_weekly,-1,-2)

[*********************100%***********************]  1 of 1 completed


YF   : [[0.01976954936981201, 0.976645827293396]]
MP   : 35665.6875

TVB  : [[0.019575268030166626, 0.9754701256752014]]
MP   : 35647.715



In [109]:
yf.download("btc-usd",period="10d",interval="15m")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-05-17 00:00:00+00:00,29862.408203,29927.445312,29844.753906,29927.445312,29927.445312,30799872
2022-05-17 00:15:00+00:00,29914.224609,30023.515625,29914.224609,29977.914062,29977.914062,51279872
2022-05-17 00:30:00+00:00,29983.738281,30066.693359,29983.509766,30066.693359,30066.693359,46886912
2022-05-17 00:45:00+00:00,30065.865234,30148.677734,30065.865234,30127.849609,30127.849609,231655424
2022-05-17 01:00:00+00:00,30135.078125,30137.910156,30003.587891,30004.783203,30004.783203,0
...,...,...,...,...,...,...
2022-05-26 13:30:00+00:00,28274.000000,28427.033203,28261.906250,28386.494141,28386.494141,561100800
2022-05-26 13:45:00+00:00,28403.406250,28816.783203,28403.406250,28811.992188,28811.992188,1196142592
2022-05-26 14:00:00+00:00,28814.392578,29132.320312,28807.746094,29132.320312,29132.320312,1025417216


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
